<a href="https://colab.research.google.com/github/atiyehghm/Generative_Models_Evaluation/blob/main/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

# data = {'Method':['BASELINE', 'CONF_1', 'CONF_2', 'CONF_3', 'CONF_4', 'CONF_ADAPTIVE', 'CONF_FIVE'],
#         'path': ['' for _ in range(7)],
#         '# of Images': [0 for _ in range(7)],
#         'Average_time': [0 for _ in range(7)],
#         'Time Variance': [0 for _ in range(7)],
#         'FID':[0 for _ in range(7)],
#         'CLIP Score': [0 for _ in range(7)],
#         'LPIPS': [0 for _ in range(7)],
#         'SSIM':[0 for _ in range(7)]}

# df = pd.DataFrame.from_dict(data)
# path = "/content/drive/MyDrive/cache experiments #1 - 9 Mordad/"
df = pd.read_csv("results.csv")
df

,Unnamed: 0,Method,path,# of Images,Average_time,Time Variance,FID,CLIP Score,LPIPS,SSIM,PSNR
0,0,BASELINE,/content/drive/MyDrive/cache experiments #1 - ...,2000,7.616406,0.001580,0,0,0,0.190395,27.909709
1,1,CONF_1,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.189115,0.025159,0,0,0,0.229638,27.901721
2,2,CONF_2,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.138027,0.001730,0,0,0,0.229165,27.902734
3,3,CONF_3,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.098629,0.001865,0,0,0,0.232253,27.904644
4,4,CONF_4,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.128788,0.001629,0,0,0,0.229880,27.910120
5,5,CONF_ADAPTIVE,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.197988,0.001246,0,0,0,0.232048,27.904214
6,6,CONF_FIVE,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.147219,0.001567,0,0,0,0.229480,27.901363


## Counting the number of files

Making sure the generation is complete

There should be 2000 images.


In [ ]:
import os, os.path

# path joining version for other paths
for m in df['Method'].values:
  DIR = path+ m + '/' + m
  df.loc[df.Method == m,'path'] = DIR
  df.loc[df.Method == m, '# of Images'] = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])

## Checking Classes

Check whether there is two samples of each class or not.

## Average Time Evaluation

In [ ]:
import cv2
import glob
import os
import numpy as np


for path in df['path'].values:
		# path = "/content/drive/MyDrive/cache experiments #1 - 9 Mordad/BASELINE/BASELINE"
		# print(df.loc[df.path == path,'Method'])
		image_paths = []
		image_extensions = ["png"]
		d = {}
		total_time = []
		for impath in glob.glob(os.path.join(path, "*.{}".format("png"))):
			image_paths.append(impath)
		for idx, impath in enumerate(image_paths):
			if image_paths[idx].find("exec_time=") >0:
				time = image_paths[idx][image_paths[idx].find("exec_time=") + 10:image_paths[idx].find("exec_time=") + 18]
				# print(float(time.replace('_', '')))
				total_time.append(float(time.replace('_', '.')))

		df.loc[df.path == path,'Average_time'] = np.mean(total_time)
		df.loc[df.path == path,'Time Variance'] = np.var(total_time)

<ipython-input-5-48307fd229a8>:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '7.616406079' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df.path == path,'Average_time'] = np.mean(total_time)
<ipython-input-5-48307fd229a8>:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.0015803567731067585' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df.path == path,'Time Variance'] = np.var(total_time)


In [ ]:
df

,Method,path,# of Images,Average_time,Time Variance,FID,CLIP Score,LPIPS,SSIM
0,BASELINE,/content/drive/MyDrive/cache experiments #1 - ...,2000,7.616406,0.001580,0,0,0,0
1,CONF_1,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.189115,0.025159,0,0,0,0
2,CONF_2,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.138027,0.001730,0,0,0,0
3,CONF_3,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.098629,0.001865,0,0,0,0
4,CONF_4,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.128788,0.001629,0,0,0,0
5,CONF_ADAPTIVE,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.197988,0.001246,0,0,0,0
6,CONF_FIVE,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.147219,0.001567,0,0,0,0


In [ ]:
df.to_csv(path+'results.csv')

## SSIM Evaluation

In [ ]:
from skimage.metrics import structural_similarity as ssim
import os
import json
import glob
import cv2
import numpy as np

def create_utils(json_dir, real_data_path):
  class2num = {}
  with open(json_dir) as json_file:
      json_data = json.load(json_file)
      for num in json_data:
          class2num[json_data[num][0]] = json_data[num][1]

  imagenet_classified = {}
  for c in class2num.keys():
      impaths = []
      for impath in glob.glob(os.path.join(real_data_path, "*{}*.jpeg".format(c))):
          # print(impath)
          impaths.append(impath)
          if len(impaths) == 2:
              imagenet_classified[c] = impaths
              break

  return class2num, imagenet_classified


def calculate_SSIM(
        class2num,
        imagenet_classified,
        generated_path):

    generated_classified = {}
    for c in class2num.keys():
        impaths = []
        for impath in glob.glob(os.path.join(generated_path, "*{}*.png".format(c))):
            impaths.append(impath)
            if len(impaths) == 2:
                generated_classified[c] = impaths
                break
    ssim_scores = []
    counter = 0
    for cid in class2num.keys():
        if cid in generated_classified.keys():
            # print(f'image number {counter}, id = {cid}')
            for i in range(2):
                real_image = cv2.imread(imagenet_classified[cid][i])
                real_image = cv2.cvtColor(real_image, cv2.COLOR_BGR2GRAY)
                generated_image = cv2.imread(generated_classified[cid][i])
                generated_image = cv2.cvtColor(generated_image, cv2.COLOR_BGR2GRAY)
                score = ssim(real_image, generated_image)
                ssim_scores.append(score)
            counter+=1

    return np.mean(ssim_scores)


json_dir = 'drive/MyDrive/ImageNet_class_index.json'
real_data_path = "drive/MyDrive/source_5k"
class2num, imagenet_classified = create_utils(json_dir, real_data_path)

for path in df['path'].values:
  if 'BASELINE' in path:
    continue
  print(df[df.path == path]['Method'])
  score = calculate_SSIM(class2num, imagenet_classified, path)
  print(f'The Average SSIM is {score}')
  df.loc[df.path == path,'SSIM'] = score

1    CONF_1
Name: Method, dtype: object
The Average SSIM is 0.22963824225110385
2    CONF_2
Name: Method, dtype: object
The Average SSIM is 0.2291649002224305
3    CONF_3
Name: Method, dtype: object
The Average SSIM is 0.23225262208020284
4    CONF_4
Name: Method, dtype: object
The Average SSIM is 0.22988026836849212
5    CONF_ADAPTIVE
Name: Method, dtype: object
The Average SSIM is 0.23204793691911307
6    CONF_FIVE
Name: Method, dtype: object
The Average SSIM is 0.22947958556384737


In [ ]:
df

,Method,path,# of Images,Average_time,Time Variance,FID,CLIP Score,LPIPS,SSIM
0,BASELINE,/content/drive/MyDrive/cache experiments #1 - ...,2000,7.616406,0.001580,0,0,0,0.190395
1,CONF_1,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.189115,0.025159,0,0,0,0.229638
2,CONF_2,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.138027,0.001730,0,0,0,0.229165
3,CONF_3,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.098629,0.001865,0,0,0,0.232253
4,CONF_4,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.128788,0.001629,0,0,0,0.229880
5,CONF_ADAPTIVE,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.197988,0.001246,0,0,0,0.232048
6,CONF_FIVE,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.147219,0.001567,0,0,0,0.229480


In [ ]:
df.to_csv('results.csv')

## PSNR Evaluation

In [ ]:
import os
import json
import glob
import cv2
import numpy as np


def create_utils(json_dir, real_data_path):
  class2num = {}
  with open(json_dir) as json_file:
      json_data = json.load(json_file)
      for num in json_data:
          class2num[json_data[num][0]] = json_data[num][1]

  imagenet_classified = {}
  for c in class2num.keys():
      impaths = []
      for impath in glob.glob(os.path.join(real_data_path, "*{}*.jpeg".format(c))):
          # print(impath)
          impaths.append(impath)
          if len(impaths) == 2:
              imagenet_classified[c] = impaths
              break

  return class2num, imagenet_classified



def calculate_PSNR(class2num,
                   imagenet_classified,
                   generated_path):

    generated_classified = {}
    for c in class2num.keys():
        impaths = []
        for impath in glob.glob(os.path.join(generated_path, "*{}*.png".format(c))):
            impaths.append(impath)
            if len(impaths) == 2:
                generated_classified[c] = impaths
                break
    psnr_values = []
    counter = 0
    for cid in class2num.keys():
        if cid in generated_classified.keys():
            # print(f'image number {counter}, id = {cid}')
            for i in range(2):
                real_image = cv2.imread(imagenet_classified[cid][i])
                real_image = cv2.cvtColor(real_image, cv2.COLOR_BGR2GRAY)
                generated_image = cv2.imread(generated_classified[cid][i])
                generated_image = cv2.cvtColor(generated_image, cv2.COLOR_BGR2GRAY)
                # Calculate MSE
                mse = np.mean((real_image - generated_image) ** 2)
                # Calculate PSNR
                if mse == 0:
                    psnr = float('inf')
                else:
                    max_pixel = 255.0
                    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
                psnr_values.append(psnr)
            counter += 1

    return np.mean(psnr_values)


json_dir = 'drive/MyDrive/ImageNet_class_index.json'
real_data_path = "drive/MyDrive/source_5k"
class2num, imagenet_classified = create_utils(json_dir, real_data_path)

for path in df['path'].values:
  print(df[df.path == path]['Method'])
  score = calculate_PSNR(class2num, imagenet_classified, path)
  print(f'The Average PSNR is {score}')
  df.loc[df.path == path,'PSNR'] = score

0    BASELINE
Name: Method, dtype: object
The Average PSNR is 27.909709001672468
1    CONF_1
Name: Method, dtype: object
The Average PSNR is 27.901720859343477
2    CONF_2
Name: Method, dtype: object
The Average PSNR is 27.90273381204057
3    CONF_3
Name: Method, dtype: object
The Average PSNR is 27.90464369548201
4    CONF_4
Name: Method, dtype: object
The Average PSNR is 27.91011997265928
5    CONF_ADAPTIVE
Name: Method, dtype: object
The Average PSNR is 27.90421393504863
6    CONF_FIVE
Name: Method, dtype: object
The Average PSNR is 27.901363000176055


In [ ]:
df

,Method,path,# of Images,Average_time,Time Variance,FID,CLIP Score,LPIPS,SSIM,PSNR
0,BASELINE,/content/drive/MyDrive/cache experiments #1 - ...,2000,7.616406,0.001580,0,0,0,0.190395,27.909709
1,CONF_1,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.189115,0.025159,0,0,0,0.229638,27.901721
2,CONF_2,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.138027,0.001730,0,0,0,0.229165,27.902734
3,CONF_3,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.098629,0.001865,0,0,0,0.232253,27.904644
4,CONF_4,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.128788,0.001629,0,0,0,0.229880,27.910120
5,CONF_ADAPTIVE,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.197988,0.001246,0,0,0,0.232048,27.904214
6,CONF_FIVE,/content/drive/MyDrive/cache experiments #1 - ...,2000,6.147219,0.001567,0,0,0,0.229480,27.901363


In [ ]:
df.to_csv('results.csv')

## LPIPS Evaluation

In [ ]:
pip install lpips

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import lpips
import torch
import json, glob, os
import cv2
import numpy as np
from torchvision import transforms


def create_utils(json_dir, real_data_path):
  class2num = {}
  with open(json_dir) as json_file:
      json_data = json.load(json_file)
      for num in json_data:
          class2num[json_data[num][0]] = json_data[num][1]

  imagenet_classified = {}
  for c in class2num.keys():
      impaths = []
      for impath in glob.glob(os.path.join(real_data_path, "*{}*.jpeg".format(c))):
          # print(impath)
          impaths.append(impath)
          if len(impaths) == 2:
              imagenet_classified[c] = impaths
              break

  return class2num, imagenet_classified

def calculate_LPIPS(
        class2num,
        imagenet_classified,
        generated_path):


    generated_classified = {}
    for c in class2num.keys():
        impaths = []
        for impath in glob.glob(os.path.join(generated_path, "*{}*.png".format(c))):
            impaths.append(impath)
            if len(impaths)==2:
                generated_classified[c] = impaths
                break

    loss_fn_alex = lpips.LPIPS(net='alex')
    print(len(generated_classified), len(imagenet_classified))
    LPIPS_scores = []
    counter = 0
    convert_tensor = transforms.ToTensor()
    for cid in class2num.keys():
        if cid in generated_classified.keys():
            # print(f'image number {counter}, id = {cid}')
            for i in range(2):
                real_image = cv2.imread(imagenet_classified[cid][i])
                real_image = cv2.cvtColor(real_image, cv2.COLOR_BGR2RGB)
                real_image = real_image.astype('float32') / 255.0
                real_image = (real_image - 0.5) / 0.5
                img0 = torch.from_numpy(real_image).permute(2, 0, 1).unsqueeze(0)
                generated_image = cv2.imread(generated_classified[cid][i])
                generated_image = cv2.cvtColor(generated_image, cv2.COLOR_BGR2RGB)
                generated_image = generated_image.astype('float32') / 255.0
                generated_image = (generated_image - 0.5) / 0.5
                img1 = torch.from_numpy(generated_image).permute(2, 0, 1).unsqueeze(0)
                d = loss_fn_alex(img0, img1)
                LPIPS_scores.append(d)
            counter+=1

    return np.mean(LPIPS_scores)

json_dir = 'drive/MyDrive/ImageNet_class_index.json'
real_data_path = "drive/MyDrive/source_5k"
class2num, imagenet_classified = create_utils(json_dir, real_data_path)

for path in df['path'].values:
  print(df[df.path == path]['Method'])
  score = calculate_LPIPS(class2num, imagenet_classified, path)
  print(f'The Average LPIPS is {score}')
  df.loc[df.path == path,'LPIPS'] = score

0    BASELINE
Name: Method, dtype: object
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth
1000 1000


In [ ]:
df

In [ ]:
df.to_csv('results.csv')

## FID Evaluation


In [ ]:
! pip install pytorch-fid

In [ ]:
#BASELINE --> 33.66245183445534

In [ ]:
!python -m pytorch_fid "drive/MyDrive/source_5k" "/content/drive/MyDrive/cache experiments #1 - 9 Mordad/BASELINE/BASELINE"

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:00<00:00, 133MB/s]
100% 100/100 [02:51<00:00,  1.71s/it]
100% 40/40 [01:25<00:00,  2.14s/it]
FID:  33.66245183445534


## CLIP Score


In [ ]:
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
!pip install clip-score

In [ ]:
## creating txt files for each image
import cv2
import glob
import os
import numpy as np

image_path = "/content/drive/MyDrive/cache experiments #1 - 9 Mordad/CONF_ADAPTIVE/CONF_ADAPTIVE"
text_path = "/content/drive/MyDrive/cache experiments #1 - 9 Mordad/TEXTS"

image_paths = []
image_extensions = ["png"]
d = {}
for impath in glob.glob(os.path.join(image_path, "*.{}".format("png"))):
	image_paths.append(impath)

i = 0
for idx, impath in enumerate(image_paths):
  if image_paths[idx].find("class_name=") >0:
    # print(i)
    file_name= image_paths[idx][len(image_path):-4]
    start_idx = file_name.find("class_name=")
    end_idx = file_name[start_idx:].find("class_id=")
    class_name = file_name[start_idx+11: end_idx-1].replace('_', ' ')
    text_name = text_path + file_name +'.txt'
    # print(text_name)
    file = open(text_name, 'w')
    file.write(f"A high quality photograph of a {class_name}")
    file.close()
    i+= 1

In [ ]:
!python -m clip_score "/content/drive/MyDrive/cache experiments #1 - 9 Mordad/CONF_ADAPTIVE/CONF_ADAPTIVE" "/content/drive/MyDrive/cache experiments #1 - 9 Mordad/TEXTS"

Loading CLIP model: ViT-B/32
100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 149MiB/s]
Calculating CLIP Score:
100% 40/40 [11:43<00:00, 17.58s/it]
CLIP Score:  30.99700164794922


In [ ]:
!rm -rf "/content/drive/MyDrive/cache experiments #1 - 9 Mordad/TEXTS"
!mkdir "/content/drive/MyDrive/cache experiments #1 - 9 Mordad/TEXTS"

In [ ]:
## BASELINE --> 31.004520416259766
## CONF_1 --> 30.833478927612305
## CONF_2 --> 30.975624084472656
## CONF_3 --> 31.059921264648438
## CONF_4 --> 30.970884323120117
## CONF_FIVE --> 31.029335021972656
## CONF_ADAPTIVE --> 30.99700164794922

In [ ]:
 import os

 DIR = "/content/drive/MyDrive/cache experiments #1 - 9 Mordad/CONF_ADAPTIVE/CONF_ADAPTIVE"
 len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/cache experiments #1 - 9 Mordad/CONF_ADAPTIVE/CONF_ADAPTIVE'

#### How to interpret CLIP score

When interpreting CLIP scores, keep in mind the following:

*  **Higher scores indicate stronger similarity**: A higher CLIP
score implies that the image and text are more similar or closely related.

*  **Scores close to zero indicate no similarity**: A CLIP score close to zero suggests that the image and text are not similar or are unrelated.

*  **Negative scores indicate dissimilarity**: A negative CLIP score implies that the image and text are dissimilar or contradict each other.

## MACs and FLOPs

### diffusion MACs

In [ ]:
!pip install --upgrade calflops

In [ ]:
pip install diffusers transformers

In [ ]:
import torch
from calflops import calculate_flops
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import StableDiffusionPipeline

/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


In [ ]:
## Evaluation of diffusion without merging
model_id = "runwayml/stable-diffusion-v1-5"
pipeline = StableDiffusionPipeline.from_pretrained(model_id)
pipeline = pipeline.to("cuda")

# You can customize the input size based on your preference
batch_size = 2
height = 512
width = 512

# Generate a dummy text input
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_input = tokenizer(
    ["An example prompt"] * batch_size,
    padding="max_length",
    max_length=tokenizer.model_max_length,
    return_tensors="pt"
)
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").to("cuda")
text_embeddings = text_encoder(text_input.input_ids.to("cuda"))[0]

# Generate a dummy latents input
num_channels = 4
latent_shape = (batch_size, num_channels, height // 8, width // 8)
latents = torch.randn(latent_shape, device="cuda")
inputs = {}
inputs["sample"] = latents
inputs["timestep"] = torch.tensor(1.0).to("cuda")
inputs["encoder_hidden_states"] = text_embeddings

# Calculate FLOPs for the U-Net model
def get_unet_flops(model):
    flops = calculate_flops(model.unet, kwargs=inputs, print_detailed=False)
    return flops

# # Calculate FLOPs for the CLIP Text Encoder
# def get_text_encoder_flops(model):
#     flops = calculate_flops(model.text_encoder, (text_input.input_ids.to("cuda"),))
#     return flops

# # Get the FLOPs
unet_flops = get_unet_flops(pipeline)
# # text_encoder_flops = get_text_encoder_flops(pipeline)

print(f"UNet FLOPs: {unet_flops}")
# print(f"Text Encoder FLOPs: {text_encoder_flops}")
# print(f"Total FLOPs: {unet_flops + text_encoder_flops}")

/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.w

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]


------------------------------------- Calculate Flops Results -------------------------------------
Notations:
number of parameters (Params), number of multiply-accumulate operations(MACs),
number of floating-point operations (FLOPs), floating-point operations per second (FLOPS),
fwd FLOPs (model forward propagation FLOPs), bwd FLOPs (model backward propagation FLOPs),
default model backpropagation takes 2.00 times as much computation as forward propagation.

Total Training Params:                                                  859.52 M
fwd MACs:                                                               677.22 GMACs
fwd FLOPs:                                                              1.36 TFLOPS
fwd+bwd MACs:                                                           2.03 TMACs
fwd+bwd FLOPs:                                                          4.07 TFLOPS
---------------------------------------------------------------------------------------------------
UNet FLOPs: ('1.36

### ToMe MACs

In [ ]:
## Evaluation of diffusion with merging
! pip install tomesd

In [ ]:
import tomesd
# Load the model
# pipe = StableDiffusionPipeline.from_pretrained('runwayml/stable-diffusion-v1-5').to("cuda:0")
# pipe.safety_checker = None
# tomesd.apply_patch(pipe, ratio=0.5)

batch_size = 2
height = 512
width = 512
num_channels = 4

text_input = pipe.tokenizer(
    ["An example prompt"] * batch_size,
    padding="max_length",
    max_length=pipe.tokenizer.model_max_length,
    return_tensors="pt"
)
text_embeddings = pipe.text_encoder(text_input.input_ids.to("cuda"))[0]
latent_shape = (batch_size, num_channels, height // 8, width // 8)
latents = torch.randn(latent_shape, device="cuda")
inputs = {}
inputs["sample"] = latents
inputs["timestep"] = torch.tensor(1.0).to("cuda")
inputs["encoder_hidden_states"] = text_embeddings

# # Calculate FLOPs for the U-Net model
flops = calculate_flops(pipe.unet, kwargs=inputs, print_detailed=False)
print(f"UNet FLOPs: {unet_flops}")

IndexError: tuple index out of range

In [ ]:
import torch, tomesd, time
from diffusers import StableDiffusionPipeline
from torch.profiler import ProfilerActivity, profile


pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")
tomesd.apply_patch(pipe, ratio=0.5)
input_text = "a photo of an astronaut riding a horse on mars"


with profile(activities=[ProfilerActivity.GPU], record_shapes=True, use_cuda=True) as prof:
    image = pipe(input_text).images[0]


print(prof.key_averages().table(sort_by="cuda_time_total"))
prof.export_chrome_trace("./trace.json")



Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/profiler/profiler.py:599: UserWarning: use_cuda is deprecated, use activities argument instead
  warn("use_cuda is deprecated, use activities argument instead")


  0%|          | 0/50 [00:00<?, ?it/s]

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
void cudnn::ops::nchwToNhwcKernel<__half, __half, fl...         0.00%       0.000us         0.00%       0.000us       0.000us        1.154s        18.90%        1.154s     210.937us          5472  
sm75_xmma_fprop_implicit_gemm_f16f16_f16f32_f32_nhwc...         0.00%       0.000us         0.00%       0.000us       0.000us     595.686ms         9.75%     595.686ms     881.192us           676  
sm75_xmma